In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'

# Importing required libraries
import sys
import time
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import ast
import pyspark.sql as ps

# function to send recieved messages from producer to db
def sendDataToDB(iter):
    client = MongoClient() #initialising client
    db = client.fit5148_db # creating Db
    climate = db.climate # Creating collection
    i = 0
    hotspot_list =[]
    climate_list = []
    for record in iter:
        data = ast.literal_eval(record[1]) # Converts the input message to dictionary
        if data["id"]=='sender_1':
            climate_list.append(data)    # creating two lists for climate and hotspot records        
        else:
            hotspot_list.append(data)
    # Algorithm :  We will check if the stream has more than one hotspot record, if yes we will check if the records 
    # have same geohash, if yes we will average surface temperature and confidence and append only one record to db. If no, we 
    # will discard the hotspot record.But we will update the climate record.
    if(len(hotspot_list)>1): 
        if(hotspot_list[0]['geohash']!=hotspot_list[1]['geohash']):
         
            for i in range(len(hotspot_list)):
                for j in range(len(climate_list)):
                    k = hotspot_list[i]['geohash']
                    l = climate_list[j]['geohash']
                    if(k == l):
                        climate_list[j]['fire_record'] = hotspot_list[i]
                        #hotspot.insert_one(hotspot_list[i])
                        break
        else:
            x= {}
            y = 0
            k = 0
            #if equal geohash,averaging confidence & surface_temperature
            for i in range(len(hotspot_list)):
                y = y +int(hotspot_list[i]['confidence'])
                k = k + int(hotspot_list[i]['surface_temperature_celcius'])

            x['confidence']=y/2
            x['surface_temperature_celcius']=k/2
            x['latitude'] =hotspot_list[0]['latitude']
            x['longitude'] = hotspot_list[0]['longitude']
            for j in range(len(climate_list)):
                    k = hotspot_list[0]['geohash']
                    l = climate_list[j]['geohash']
                    if(k == l):
                        climate_list[j]['fire_record'] = x
                        break
    #Checking of it has only one record and matching geo hash key generated                  
    elif(len(hotspot_list) == 1):
        for j in range(len(climate)):
            k = hotspot_list[0]['geohash']
            l = climate_list[j]['geohash']
            if(k == l):
                climate_list[j]['fire_record'] = hotspot_list[0]
                break
    #finally inserting the records into Db            
    try:
        climate.insert_many(climate_list)
        hotspot_list = []
        climate_list = []
    except Exception as ex:
        print("Exception Occured. Message: {0}".format(str(ex)))
                    
    client.close()

    

n_secs = 10 # proving the batch size as 10
topic = "Scenario01" # topicname same as in producers

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")  # setting two local threads
sc = SparkContext.getOrCreate() # creating spark context
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs) #initialising the streaming context
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week11-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary

lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
time.sleep(900) # Run stream for 15 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)

KeyboardInterrupt: 